In [ ]:
import neat
import threading
import flappy_multi
import numpy as np
import time

In [ ]:
class Agent :
    """Classe représentant un oiseau utilisant un réseau de neurone"""
    s = 1
    def __init__(self, genome, config) :
        """Construit le réseau de neurone à partir du genome"""
        self.net = neat.nn.FeedForwardNetwork.create(genome, config)

    def shouldJump(self, v) :
        """A partir d'un vecteur réprésentant : 
        v[0] : la hauteur du prochai tuyau
        v[1] : la hauteur de l'oiseau
        v[2] : la distance au tuyau"""
        self.s += 1
        self.s -= min(1,(abs(v[0] - v[1])/70)**0.5)
        
        v = np.array(v)*np.array([0.008,0.008, 0.02])
        input = (v[0], v[1], v[2])
        output = self.net.activate(input)
        return output[0] > 0.5
    
    def passedPip(self) :
        """Si l'oiseau passe un tuyau"""
        self.s += 100
    
    def score(self) :
        return self.s

In [ ]:
import random
class randomAgent() :
    """Agent aléatoir permettant d'avoir 10 agents dans une simulation s'il manque des agents NEAT"""
    s = 11
    def shouldJump(self, v) :
        return random.random() < 0.1
    def passedPip(self) :
        self.s += 100

In [ ]:
generation = []
lockCalculationDone = threading.Lock()
lockGenerationReady = threading.Lock()
lockGenerationReady.acquire()
lockCalculationDone.acquire()

In [ ]:
currentAgent = 0
genCount = 0
fitnessCalculated = False
def nextAgent():
    """Fonction appelée par flappyBird pour avoir un nouvel agent"""
    global currentAgent
    if currentAgent == 70 :
        global genCount
        genCount += 1
        print("Generation " + str(genCount))
        currentAgent = 0
        lockCalculationDone.release()
    if currentAgent == 0 :
        lockGenerationReady.acquire()
    #On complète avec des agents aléatoirs si besoin 
    if currentAgent >= len(generation) :
        currentAgent += 1
        return randomAgent()
    agent = generation[currentAgent]
    currentAgent += 1

    return agent

In [ ]:
def eval_genomes(genomes, config):
    """Fonction appelée par le module NEAT pour tester une génération de génome"""
    generation.clear()
    for genome_id, genome in genomes:
        generation.append(Agent(genome,config))
    lockGenerationReady.release()
    lockCalculationDone.acquire()
    
    for i,g in enumerate(genomes) :
        genome_id, genome = g
        genome.fitness = generation[i].score()

In [ ]:
def neatAlgo():
    winner = p.run(eval_genomes, 50)

In [ ]:
#Charge la config
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, "config")

In [ ]:
p = neat.Population(config)

In [ ]:
threading.Thread(target=neatAlgo).start()

In [ ]:
flappy_multi.main(nextAgent)